In [2]:
!pip install hazm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.7 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=0dd32f6b90d1e519b6c9cbea0c08ed2f427a8cdf85dc02dc88f572e6569ca479
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [36]:
from hazm import word_tokenize , Normalizer
import pandas as pd
import numpy as np                                #بازنمایی (بردار سازی)
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

#لیست سند ها

In [21]:
document=["ماه آسمان روشن است ",


          "آسمان روشن شد",

          "ماه و ستاره ها زیبا هستند",

          "اسب خواب است"]

#پیش پردازش متن

In [25]:
#normalize کردن
normalizer=Normalizer()

#تابع پیش پردازش متن
def preprocess(text):
  #نرمال سازی متن
  text=normalizer.normalize(text)
  #حذف علایم نگارشی و توکن سازی
  tokens=word_tokenize(text)

  return ' '.join(tokens)


#پیش پردازش چملات
documents= [preprocess(doc) for doc in document]

In [28]:
documents

['ماه آسمان روشن است',
 'آسمان روشن شد',
 'ماه و ستاره\u200cها زیبا هستند',
 'اسب خواب است']

#بردار تک نمود (one-hot)

In [26]:
all_tokens=set()
for doc in documents:
  tokens=doc.split()
  all_tokens.update(tokens)


one_hot_dict= dict()

for idx,word in enumerate(all_tokens):
  one_hot_vector=[0] * len(all_tokens)
  one_hot_vector[idx]=1
  one_hot_dict[word]= one_hot_vector
#اینکارو با کتابخونه هم میشه انجام داد ولی ما با کد پایتون انجام دادیم

print("دیکشنری بردار های تک نمود: ")
for word , vector in one_hot_dict.items():
  print(f"{word}: {vector}")

دیکشنری بردار های تک نمود: 
و: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ماه: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
شد: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
اسب: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
زیبا: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
ستاره‌ها: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
روشن: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
آسمان: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
خواب: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
است: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
هستند: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [27]:
one_hot_dict['اسب']

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

#ماتریس باهم‌ایی کلمات

In [30]:
#برای ایجاد ماتریس هم‌ایی استفاده از countVectorizer
vectorizer=CountVectorizer()
X= vectorizer.fit_transform(documents)

#ایجاد ماتریس هم ایی
Xc= (X.T * X)
Xc.setdiag(0)

Xc.shape[0]

11

In [31]:
df= pd.DataFrame(data=Xc.toarray(),columns=vectorizer.get_feature_names_out(),
                 index= vectorizer.get_feature_names_out())

#toarray=تبدیل به ارایه کن
#get_feature_names_out= میگه که برو ویکتورایز فیچر نیم ها اون کلمات مثل اسب و اینها
print(df)

       آسمان  اسب  است  خواب  روشن  زیبا  ستاره  شد  ماه  ها  هستند
آسمان      0    0    1     0     2     0      0   1    1   0      0
اسب        0    0    1     1     0     0      0   0    0   0      0
است        1    1    0     1     1     0      0   0    1   0      0
خواب       0    1    1     0     0     0      0   0    0   0      0
روشن       2    0    1     0     0     0      0   1    1   0      0
زیبا       0    0    0     0     0     0      1   0    1   1      1
ستاره      0    0    0     0     0     1      0   0    1   1      1
شد         1    0    0     0     1     0      0   0    0   0      0
ماه        1    0    1     0     1     1      1   0    0   1      1
ها         0    0    0     0     0     1      1   0    1   0      1
هستند      0    0    0     0     0     1      1   0    1   1      0


ذخیره در فایل اکسل مثلن برای موقعی که مقادیر عوض نشدن دیگه این مراحل رو طی نکنیم از این اکسل استفاده کنیم

In [ ]:
# df.to_excel("word_cooccurrence_matrix.xlsx", index=True)

#بسته کلمات

In [33]:
bow_df= pd.DataFrame(data=X.toarray(),columns=vectorizer.get_feature_names_out())

print(bow_df)

   آسمان  اسب  است  خواب  روشن  زیبا  ستاره  شد  ماه  ها  هستند
0      1    0    1     0     1     0      0   0    1   0      0
1      1    0    0     0     1     0      0   1    0   0      0
2      0    0    0     0     0     1      1   0    1   1      1
3      0    1    1     1     0     0      0   0    0   0      0


In [ ]:
# bow_df.to_excel("bag_of_word_matrix.xlsx", index=Flase)

#TF-IDF بازنمایی

In [38]:
tfide_vectorizer=TfidfVectorizer()
tfide_matrix=tfide_vectorizer.fit_transform(documents)

tfide_df=pd.DataFrame(data=tfide_matrix.toarray(),
                      columns=tfide_vectorizer.get_feature_names_out())

print(tfide_df)

      آسمان       اسب       است      خواب  ...        شد       ماه        ها     هستند
0  0.500000  0.000000  0.500000  0.000000  ...  0.000000  0.500000  0.000000  0.000000
1  0.526405  0.000000  0.000000  0.000000  ...  0.667679  0.000000  0.000000  0.000000
2  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.366739  0.465162  0.465162
3  0.000000  0.617614  0.486934  0.617614  ...  0.000000  0.000000  0.000000  0.000000

[4 rows x 11 columns]


In [ ]:
# df.to_excel("tfide_matrix.xlsx", index=False)

ارزش و کارایی پیش پردازش مهمه